![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [1]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [1]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [1]:
df_train.info()

In [1]:
df_train.head(5)

In [1]:
df_test.info()

In [1]:
df_test.head(5)

In [1]:
sample_submission.head(5)

In [1]:
sample_submission.info()

In [1]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [1]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [1]:
data.sample(5)

In [1]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

Let's check all columns with nulls:

#Cuisine Style        38410 non-null object

#Price Range          32639 non-null object

#Number of Reviews    46800 non-null float64

#Reviews              49998 non-null object

In [1]:

data['Cuisine Style'].isna().sum()

In [1]:
data['Cuisine Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')

In [1]:
# Добавим новые столбцы категориальные для различного ценового диапазона, рестораны без данных по цене будут иметь во всех трех колонках
# дешевый, средний, дорогой по нулю.
data['Price Range']=data['Price Range'].fillna(value = '0')


In [1]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [1]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

In [1]:
#Let's check rows with reviews == Null. Looks like Restaurants just opened... 
#or there is no enought information or some problems with data entering
data[data['Reviews'].isna()]

In [1]:
data['Reviews'] = data['Reviews'].fillna('[[], []]')

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [1]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [1]:
def assign_country (city):
    countries = {"London": 'England',
            'Paris':'France1', 
            'Madrid': 'France1',
            'Barcelona': 'Spain1',
            'Berlin': 'Germany1',
            'Milan': 'Italy1',
            'Rome': 'Italy1',
            'Prague': 'Czech_Republic1',
            'Lisbon': 'Portugal1',
            'Vienna':  'Austria1',
            'Amsterdam': 'Netherlands1',
            'Brussels' : 'Belgium1',
            'Hamburg' : 'Germany1',
            'Munich': 'Germany1',
            'Lyon' : 'France1',
            'Stockholm': 'Sweden1',
             'Budapest': 'Hungary1',
             'Warsaw': 'Poland1',
             'Dublin' : 'Ireland1',
             'Copenhagen': 'Denmark1',
             'Athens': 'Greek1',
             'Edinburgh': 'Scotland1',
             'Zurich':'Switzerland1',
             'Oporto':'Portugal',
             'Geneva':'Switzerland1',
             'Krakow':'Polish1',
             'Oslo': 'Norway1',
             'Helsinki': 'Finland1',
             'Bratislava': 'Slovakia1',
             'Luxembourg': 'Luxembourg1',
             'Ljubljana': 'Slovenia1'}
    return countries[city]

In [1]:
data['Country'] = data['City'].apply(assign_country)
country_df = pd.get_dummies(data.Country)
data= data.join(country_df)
data = pd.get_dummies(data, columns=[ 'Country',], dummy_na=True)

In [1]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [1]:
data.head(5)

In [1]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [1]:
data['Price Range'].value_counts()


По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [1]:
price_df = pd.get_dummies(data['Price Range'])
data['cheap']=price_df['$']
data['average'] = price_df['$$ - $$$']
data['expensive'] = price_df['$$$$']
data['no_data_price'] = price_df['0']
data['Price Range'].isna().sum()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

#Cuisines

In [1]:
data['Cuisine Style'] = data['Cuisine Style'].str.replace('[', '')
data['Cuisine Style'] = data['Cuisine Style'].str.replace(']', '')
data['Cuisine Style'] = data['Cuisine Style'].str.replace("'", '')
data['Cuisine Style'] = data['Cuisine Style'].str.replace(" ", '')
data['Cuisine Style'] = data['Cuisine Style'].str.split(',')
data['Cuisine Style'] = data['Cuisine Style'].fillna(value=' ')
data['cuisine_numbers'] = data['Cuisine Style'].apply(lambda x: len(x))

In [1]:
#Created dummy values for cuisines, joint with data

mlb = MultiLabelBinarizer()
cuisines_df = pd.DataFrame(mlb.fit_transform(data['Cuisine Style']),columns=mlb.classes_)
data= data.join(cuisines_df)
data

In [1]:
data['Reviews'].isna().sum()

## Reviews

In [1]:
import re
from datetime import datetime

pattern = re.compile("\d+\/\d+\/\d+")

dates = data['Reviews'].apply(pattern.findall)

data['Date_of_Reviews1'] = pd.to_datetime(dates.apply(lambda x: x[0] if len(x) > 0 else None))
data['Date_of_Reviews2'] = pd.to_datetime(dates.apply(lambda x: x[1] if len(x) > 1 else None))

# display(data.loc[:, ['date1', 'date2']].sample(5))
# data['Date_of_Reviews'] = data['Reviews'].apply(pattern.findall)
# data['Date_of_Reviews1']=pd.to_datetime(data['Date_of_Reviews'].str[0])
# data['Date_of_Reviews2']=pd.to_datetime(data['Date_of_Reviews'].str[1])
data['Delta_time'] = (data['Date_of_Reviews1']-data['Date_of_Reviews2'])
data['Delta_time'] = pd.to_numeric(data['Delta_time'].dt.days, downcast='integer')

In [1]:
# defined function to determine the emotion based on the words in the reviews (MAE decreased on 0.02)
import string
from collections import Counter

import matplotlib.pyplot as plt

def emotions(text):

    # reading reviews from dataset

    # converting to lowercase
    lower_case = text.lower()

    # Removing punctuations
    cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

#     # splitting text into words
    tokenized_words = cleaned_text.split()

    stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                  "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                  "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
                  "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
                  "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
                  "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
                  "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
                  "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
                  "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
                  "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

#         # Removing stop words from the tokenized words list
    final_words = []
    for word in tokenized_words:
        if word not in stop_words:
            final_words.append(word)

#     # NLP Emotion Algorithm
#     # 1) Check if the word in the final word list is also present in emotion.txt
#     #  - open the emotion file
#     #  - Loop through each line and clear it
#     #  - Extract the word and emotion using split

#     # 2) If word is present -> Add the emotion to emotion_list
#     # 3) Finally count each emotion in the emotion list

    emotion_list = []
    with open('../input/emotionstxt/emotions.txt', 'r') as file:
        for line in file:
            clear_line = line.replace("\n", '').replace(",", '').replace("'", '').strip()
            word, emotion = clear_line.split(':')

            if word in final_words:
                emotion_list.append(emotion)

    
#     w = Counter(emotion_list)- not used
    return (emotion_list)

In [1]:
#added emotions column to the dataframe
data['emotion']=data["Reviews"].apply(emotions)

In [1]:
#created dummy variables from the list of emotions
mlb = MultiLabelBinarizer()
emotion_df = pd.DataFrame(mlb.fit_transform(data['emotion']),columns=mlb.classes_)
data= data.join(emotion_df)

In [1]:
data.info()

### EDA

### Посмотрим распределение признака

In [1]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [1]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [1]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [1]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [1]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [1]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [1]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [1]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [1]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [1]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Cuisine Style_isNAN'] = pd.isna(df_output['Cuisine Style']).astype('uint8')
    df_output['Price Range']=df_output['Price Range'].fillna(value = '0')
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Reviews'] = df_output['Reviews'].fillna('[[], []]')
 


    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    
    city_population = {'Paris':13024, 'Stockholm':2391, 'London':14257, 
                   'Berlin':6144, 'Munich':5991, 'Oporto':1721,
                   'Milan':4336, 'Bratislava':659, 'Vienna':2600, 
                   'Rome':4342, 'Barcelona':5474, 'Madrid':6791,
                   'Dublin':1417, 'Brussels':2500, 'Zurich':415, 
                   'Warsaw':3100, 'Budapest':3011, 'Copenhagen':2057,
                   'Amsterdam':2480, 'Lyon':2323, 'Hamburg':5107, 
                   'Lisbon':2827, 'Prague':2677, 'Oslo':1588,
                   'Helsinki':1525, 'Edinburgh':901, 'Geneva':201, 
                   'Ljubljana':537, 'Athens':2928,'Luxembourg':633, 
                   'Krakow':1752}
    #number of rests in the City & people population
    
    rest_per_city = df_output.City.value_counts().to_dict()
    df_output['Ranking_Rest_Num'] = df_output.apply(lambda r: (r['Ranking'] / rest_per_city[r.City]), axis=1)
    df_output['Ranking_Population'] = df_output.apply(lambda r: (r['Ranking'] / city_population[r.City]), axis=1)
    
    countries = {"London": 'England',
                'Paris':'France1', 
                'Madrid': 'France1',
                'Barcelona': 'Spain1',
                'Berlin': 'Germany1',
                'Milan': 'Italy1',
                'Rome': 'Italy1',
                'Prague': 'Czech_Republic1',
                'Lisbon': 'Portugal1',
                'Vienna':  'Austria1',
                'Amsterdam': 'Netherlands1',
                'Brussels' : 'Belgium1',
                'Hamburg' : 'Germany1',
                'Munich': 'Germany1',
                'Lyon' : 'France1',
                'Stockholm': 'Sweden1',
                 'Budapest': 'Hungary1',
                 'Warsaw': 'Poland1',
                 'Dublin' : 'Ireland1',
                 'Copenhagen': 'Denmark1',
                 'Athens': 'Greek1',
                 'Edinburgh': 'Scotland1',
                 'Zurich':'Switzerland1',
                 'Oporto':'Portugal',
                 'Geneva':'Switzerland1',
                 'Krakow':'Polish1',
                 'Oslo': 'Norway1',
                 'Helsinki': 'Finland1',
                 'Bratislava': 'Slovakia1',
                 'Luxembourg': 'Luxembourg1',
                 'Ljubljana': 'Slovenia1'}
    def assign_country (city):
        return countries[city]
    
    df_output['Country'] = df_output['City'].apply(assign_country)
    country_df = pd.get_dummies(df_output.Country)
    df_output= df_output.join(country_df)
    df_output = pd.get_dummies(df_output, columns=[ 'Country',], dummy_na=True)
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    price_df = pd.get_dummies(df_output['Price Range'])
    df_output['cheap']=price_df['$']
    df_output['average'] = price_df['$$ - $$$']
    df_output['expensive'] = price_df['$$$$']
    df_output['no_df_output_price'] = price_df['0']
    
    df_output['Cuisine Style'] = df_output['Cuisine Style'].str.replace('[', '')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].str.replace(']', '')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].str.replace("'", '')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].str.replace(" ", '')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].str.split(',')
    df_output['Cuisine Style'] = df_output['Cuisine Style'].fillna(value=' ')
    df_output['cuisine_numbers'] = df_output['Cuisine Style'].apply(lambda x: len(x))
    
    mlb = MultiLabelBinarizer()
    cuisines_df = pd.DataFrame(mlb.fit_transform(df_output['Cuisine Style']),columns=mlb.classes_)
    df_output= df_output.join(cuisines_df)
    
    import re
    from datetime import datetime

    pattern = re.compile("\d+\/\d+\/\d+")

    dates = df_output['Reviews'].apply(pattern.findall)

    df_output['Date_of_Reviews1'] = pd.to_datetime(dates.apply(lambda x: x[0] if len(x) > 0 else None))
    df_output['Date_of_Reviews2'] = pd.to_datetime(dates.apply(lambda x: x[1] if len(x) > 1 else None))
    df_output['Delta_time'] = (df_output['Date_of_Reviews1']-df_output['Date_of_Reviews2'])
    df_output['Delta_time'] = pd.to_numeric(df_output['Delta_time'].dt.days, downcast='integer')
    df_output.drop(['Date_of_Reviews1', 'Date_of_Reviews2'], inplace=True, axis=1)
    
    import string
    from collections import Counter

    import matplotlib.pyplot as plt

    def emotions(text):

        # reading reviews from dataset

        # converting to lowercase
        lower_case = text.lower()

        # Removing punctuations
        cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

    #     # splitting text into words
        tokenized_words = cleaned_text.split()

        stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                      "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                      "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
                      "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
                      "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
                      "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
                      "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
                      "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
                      "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
                      "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

    #         # Removing stop words from the tokenized words list
        final_words = []
        for word in tokenized_words:
            if word not in stop_words:
                final_words.append(word)

    #     # NLP Emotion Algorithm
    #     # 1) Check if the word in the final word list is also present in emotion.txt
    #     #  - open the emotion file
    #     #  - Loop through each line and clear it
    #     #  - Extract the word and emotion using split

    #     # 2) If word is present -> Add the emotion to emotion_list
    #     # 3) Finally count each emotion in the emotion list

        emotion_list = []
        with open('../input/emotionstxt/emotions.txt', 'r') as file:
            for line in file:
                clear_line = line.replace("\n", '').replace(",", '').replace("'", '').strip()
                word, emotion = clear_line.split(':')

                if word in final_words:
                    emotion_list.append(emotion)


        #     w = Counter(emotion_list)- not used
            return (emotion_list)
    
    df_output['emotion']=df_output["Reviews"].apply(emotions)
    mlb = MultiLabelBinarizer()
    emotion_df = pd.DataFrame(mlb.fit_transform(df_output['emotion']),columns=mlb.classes_)
    df_output= df_output.join(emotion_df)
    
    

    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    df_output.Delta_time.fillna(0, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [1]:
df_preproc = preproc_data(data)
df_preproc.isna().sum()

In [1]:
df_preproc.info()

In [1]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [1]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [1]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [1]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [1]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [1]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)
y_pred =  (y_pred * 2).round(0) / 2

In [1]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [1]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [1]:
test_data.sample(10)

In [1]:
test_data = test_data.drop(['Rating'], axis=1)

In [1]:
sample_submission

In [1]:
predict_submission = model.predict(test_data)

In [1]:
predict_submission

In [1]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
